# Setup

Please make sure to setup the DuckDB database and tables using the below command. It creates the tables and loads in data and can take a few minutes to complete.

**NOTE**: If you have run `setup.py` already you can skip this step.


In [ ]:
! pwd

In [ ]:
! python ../../setup.py

Ensure that you are able to run SQL queries

In [ ]:
# Load the extension
%load_ext sql

In [ ]:
# Connect to DuckDB
%sql duckdb:///../../tpch.db

In [ ]:
%config SqlMagic.displaylimit = None

In [ ]:
%%sql
-- Run a simple show tables
SELECT
  table_name
FROM
  information_schema.tables
WHERE
  table_schema = 'main'

# Data Model

Through out this workshop we will use the TPCH data to run queries.

The TPC-H data is usually used to benchmark database performance. The TPC-H data represents a car parts seller’s data warehouse, where we record orders, items that make up that order (lineitem), supplier, customer, part (parts sold), region, nation, and partsupp (parts supplier). 

**Note**: Have a copy of the data model(table schemas and how they relate to each other) as you follow along; this will help in understanding the examples provided and in answering exercise questions.


![](../../images/tpch_erd.png)


# WINDOW FUNCTIONS
========================================================================================

# Use window functions to do looping with SQL

The reason for window functions is because historically, SQL has been set based i.e supporting operating on a set of rows at a time (e.g. GROUP BY, JOINS, etc).


# [WHY] Common scenarios when you want to use window functions

1. Calculate running metrics/sliding window over rows in the table (aggregate functions)
2. Ranking rows based on values in column(s) (ranking functions)
3. Access other row's values while operating on the current row (value functions)
4. Any combination of the above
 

# [HOW TO] Create a window function

## Window functions have four essential parts

1. **Partition**: Defines a set of rows based on specified column(s) value. If no partition is specified, the entire table is considered a partition.
2. **Order By**: This optional clause specifies how to order the rows within a partition.
3. **Function**: The function to be applied on the current row. The function results in an additional column in the output.
4. **Window frame**: Within a partition, a window frame allows you to specify the rows to be considered in the function computation.


![](../../images/framing.svg)


In [ ]:
%%sql
SELECT
  o_custkey,
  o_orderdate,
  o_totalprice,
  SUM(o_totalprice) -- FUNCTION 
  OVER (
    PARTITION BY
      o_custkey -- PARTITION
    ORDER BY
      o_orderdate -- ORDER BY
  ) AS running_sum
FROM
  orders
WHERE
  o_custkey = 4
ORDER BY
  o_orderdate
LIMIT
  10;

The function `SUM` that we use in the above query is an aggregate function. Notice how the `running_sum` adds up (aka aggregates) the `o_totalprice` over all the rows. The rows themselves are ordered in ascending order by its orderdate.

**Reference**: While the standard aggregate functions are `MIN, MAX, AVG, SUM, & COUNT`, modern data systems offer a variety of powerful aggregations functions. Check your database documentation for available aggreagate functions. [e.g. list of agg functions available in duckdb](https://duckdb.org/docs/sql/functions/aggregates.html#general-aggregate-functions)

# [HOW TO] Use ranking functions to get top/bottom n rows

If you are working on a problem to get the top/bottom n rows (as defined by some value) then use the **row** functions.

Let's look at an example of how to use a row function:

From the `orders` table **get the top 3 spending customers per day**. The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [ ]:
%%sql
SELECT
  *
FROM
  (
    SELECT
      o_orderdate,
      o_totalprice,
      o_custkey,
      RANK() -- RANKING FUNCTION 
      OVER (
        PARTITION BY
          o_orderdate -- PARTITION BY order date
        ORDER BY
          o_totalprice DESC -- ORDER rows withing partition by totalprice
      ) AS rnk
    FROM
      orders
  )
WHERE
  rnk <= 3
ORDER BY
  o_orderdate
LIMIT
  5;


# [REFERENCE] standard RANKING functions: 

1. **`RANK`**: Ranks the rows starting from 1 to n within the window frame. Ranks the rows with the same value (defined by the "ORDER BY" clause) as the same and skips the ranking numbers that would have been present if the values were different.

2. **`DENSE_RANK`**: Ranks the rows starting from 1 to n within the window frame. Ranks the rows with the same value (defined by the "ORDER BY" clause) as the same and does not skip any ranking numbers.

3. **`ROW_NUMBER`**: Adds a row number that starts from 1 to n within the window frame and does not create any repeating values.



Now that we have see how to **define a window function** and how to use **ranking and aggregation** functions, let's take it a step further by practicing **value functions**.

Remember that value functions are used to access other row's values while operating on the current row

## User exercise 1: 10min

* Write a SQL query using the `orders` table that calculates the following columns:
	1. o_orderdate
	2. o_custkey
	3. o_totalprice
	4. totalprice_diff: The customers current day's o_totalprice - that same customers most recent previous purchase's o_totalprice

* **Hint**: Use the `LAG(column_name)` ranking function to identify the prior day's revenue.

* The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [ ]:
%%sql
-- write your query here

# [REFERENCE] Here is a quick definition of the standard VALUE functions: 

1. **NTILE(n)**: Divides the rows in the window frame into n approximately equal groups, and assigns a number to each row indicating which group it belongs to.
2. **FIRST_VALUE()**: Returns the first value in the window frame.
3. **LAST_VALUE()**: Returns the last value in the window frame.
4. **LAG()**: Accesses data from a previous row within the window frame.
5. **LEAD()**: Accesses data from a subsequent row within the window frame.

## [WHY] define a window frame when you have a partition?

While our functions operate on the rows in the partition a window frame provides more granular ways to operate on a select set of rows within a partition.

Example

Consider a scenario where you have sales data, and you want to calculate a 3-day moving average of sales within each store:

```sql
SELECT
    store_id,
    sale_date,
    sales_amount,
    AVG(sales_amount) OVER (
        PARTITION BY store_id
        ORDER BY sale_date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS moving_avg_sales
FROM
    sales;
```

In this example:

1. **PARTITION BY** store_id ensures the calculation is done separately for each store.
2. **ORDER BY** sale_date defines the order of rows within each partition.
3. **ROWS BETWEEN 2 PRECEDING AND CURRENT ROW** specifies the window frame, considering the current row and the two preceding rows to calculate the moving average.
add: image
Without defining the window frame, the function might not be able to provide the specific moving average calculation you need.

## [HOW] to define a window frame

There are two major ways to define a window frame

1. **ROWS**: Used to select a set of rows relative to the current row based on position.
    1. Row definition format `ROWS BETWEEN start_point AND end_point`.
    2. The start_point and end_point can be any of the following three (in the proper order:
        1. **n PRECEDING**: n rows preceding the current row. UNBOUNDED PRECEDING indicates all rows before the current row.
        2. **n FOLLOWING**: n rows following the current row. UNBOUNDED FOLLOWING indicates all rows after the current row.
2. **RANGE**: Used to select a set of rows relative to the current row based on the value of the columns specified in the `ORDER BY` clause.
    1. Range definition format `RANGE BETWEEN start_point AND end_point`.
    2. The start_point and end_point can be any of the following:
        1. **CURRENT ROW**: The current row.
        2. **n PRECEDING**: All rows with values within the specified range that are less than or equal to n units preceding the value of the current row.
        3. **n FOLLOWING**: All rows with values within the specified range that are greater than or equal to n units following the value of the current row.
        4. **UNBOUNDED PRECEDING**: All rows before the current row within the partition.
        5. **UNBOUNDED FOLLOWING**: All rows after the current row within the partition.
    3. `RANGE` is particularly useful when dealing with numeric or date/time ranges, allowing for calculations like running totals, moving averages, or cumulative distributions.


## Example:


1. Write a query on the orders table that has the following output:
    1. o_custkey
    2. order_month: In YYYY-MM format, use strftime(o_orderdate, '%Y-%m') AS order_month
    3. total_price: Sum of o_totalprice for that month
    4. three_mo_total_price_avg: The 3 month (previous, current & next) average of total_price for that customer



In [ ]:
%%sql
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER ( -- FUNCTION: RUNNING AVERAGE
      PARTITION BY
        o_custkey -- PARTITIONED BY customer
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING -- WINDOW FRAME DEFINED AS 1 ROW PRECEDING to 1 ROW FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      strftime (o_orderdate, '%Y-%m') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
LIMIT
  5;

## User exercise 2: 10 min

Now that we have seen how to create a window frame with ROWS, let' explore how to do this with RANGE.

1. Write a query on the orders table that has the following output:
    1. order_month, 
    2. o_custkey,
    3. total_price,
    4. three_mo_total_price_avg
    5. **consecutive_three_mo_total_price_avg**: The consecutive 3 month average of total_price for that customer. Note that this should only include months that are chronologically next to each other.

**Hint**: Use `CAST(strftime(o_orderdate, '%Y-%m-01') AS DATE)` to cast order_month to date format
                                                                                           
* The orders table schema is shown below:

![Orders table](../../images/orders.png)

In [ ]:
%%sql
-- write your query here

# Recap

1. Use windows when:
	* Calculating running metrics (similar to GROUP BY, but keeping all the rows)
	* Rank rows based on specific column(s)
	* Access other row values from the current row
2. Window has four key parts: Partition, Order By, Function, Window Frame
3. Define window frame with ROWS or RANGE
4. Window functions are expensive; be mindful of the performance (Covered in another lesson)
